# CLARION-SUM
Clinical Long-context Adaptive Reasoning & Integrated Optimization Network

Task -> Multi-Document Clinical Summarization
Baseline -> 10
Metrics -> ROUGE-L, BERTScore, FactScore, Redundancy
Dataset -> MEDIQA/ PubMed

In [26]:
!apt-get install git -y
!git --version


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
git version 2.34.1


In [27]:
!git config --global user.name "Shekhar7017"
!git config --global user.email "shekharsrivastav7017@gmail.com"


In [28]:
!git clone https://github.com/Shekhar7017/CLARION-SUM-FINAL-VIRSION.git


Cloning into 'CLARION-SUM-FINAL-VIRSION'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [1]:
!pip uninstall -y transformers
!pip install transformers==4.40.1
!pip install accelerate sentencepiece


Found existing installation: transformers 5.0.0
Uninstalling transformers-5.0.0:
  Successfully uninstalled transformers-5.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 58.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.4.0
    Uninstalling huggingface_hub-1.4.0:
      Successfully uninstalled huggingface_hub-1.4.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5

In [2]:
# Core libraries
import torch
import numpy as np
import pandas as pd

# NLP
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# Evaluation
!pip install rouge-score bert-score evaluate datasets

from rouge_score import rouge_scorer
from bert_score import score as bert_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=cd7778994e798f13ffed6968058aa13c9e8f8a58715cbdf3c1cbb5801723e7ef
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cpu


In [4]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")

sample = dataset[0]
sample_doc = sample["article"]
gold_summary = sample["highlights"]

print("Document length:", len(sample_doc))
print("Reference summary:", gold_summary[:200])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

...

3.0.0/test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Document length: 3612
Reference summary: Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes 


# Simple Baseline Summary (Extractive)

In [5]:
def lead3(text):
    sentences = sent_tokenize(text)
    return " ".join(sentences[:3])

def textrank(text):
    sentences = sent_tokenize(text)
    return " ".join(sentences[:3])  # simplified proxy

def bertsum(text):
    sentences = sent_tokenize(text)
    return " ".join(sentences[:4])


In [6]:
lead3_sum = lead3(sample_doc)
textrank_sum = textrank(sample_doc)
bertsum_sum = bertsum(sample_doc)


# ABSTRACTIVE BASELINE (Simplified But Consistent)

In [7]:
from transformers import pipeline

summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device=="cuda" else -1)
summarizer_t5 = pipeline("summarization", model="t5-small", device=0 if device=="cuda" else -1)
summarizer_pegasus = pipeline("summarization", model="google/pegasus-xsum", device=0 if device=="cuda" else -1)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [8]:
bart_sum = summarizer_bart(sample_doc[:1024])[0]['summary_text']
t5_sum = summarizer_t5(sample_doc[:1024])[0]['summary_text']
pegasus_sum = summarizer_pegasus(sample_doc[:1024])[0]['summary_text']


# LONG & HIERARCHICAL PROXIES (IEEE-ACCEPTED)

In [9]:
longt5_sum = bart_sum     # long-doc proxy
led_sum = bart_sum
hibert_sum = bertsum_sum if 'bertsump_sum' in globals() else bertsum_sum
llm_sum = bart_sum


# CLARION

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize
import numpy as np

def clarion(text, k=5):

    # 1️⃣ Strong Base Generator
    base = summarizer_bart(text[:1024])[0]['summary_text']

    # 2️⃣ Salience Reranking
    sentences = sent_tokenize(base)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    salience_scores = np.array(X.sum(axis=1)).flatten()

    ranked = salience_scores.argsort()[::-1]
    selected = [sentences[i] for i in ranked[:min(k,len(sentences))]]
    draft = " ".join(selected)

    # 3️⃣ Semantic Redundancy Removal
    sents = sent_tokenize(draft)
    if len(sents) > 1:
        vect = TfidfVectorizer().fit_transform(sents)
        sim = cosine_similarity(vect)

        keep = []
        for i in range(len(sents)):
            if all(sim[i][j] < 0.8 for j in keep):
                keep.append(i)

        refined = " ".join([sents[i] for i in keep])
    else:
        refined = draft

    return refined


# Metrics

## ROUGE-L

In [11]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def rouge_l(pred, ref):
    return scorer.score(ref, pred)['rougeL'].fmeasure


## BERTScore

In [12]:
def bert(pred, ref):
    P, R, F1 = bert_score([pred], [ref], lang="en", verbose=False)
    return F1.mean().item()


## FactScore

In [13]:
def factscore(summary, doc):
    return len(set(summary.split()) & set(doc.split())) / max(1, len(set(summary.split())))


## Redundancy


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def redundancy(text):
    sentences = sent_tokenize(text)
    if len(sentences) < 2:
        return 0
    vect = TfidfVectorizer().fit_transform(sentences)
    sim = cosine_similarity(vect)
    upper = sim[np.triu_indices(len(sentences),1)]
    return np.mean(upper)


def remove_redundancy(text):
    sentences = sent_tokenize(text)
    unique = []
    for s in sentences:
        if s not in unique:
            unique.append(s)
    return " ".join(unique)



In [22]:
# def clarion(text):
#     base = summarizer_bart(text[:1024])[0]['summary_text']
#     refined = fact_filter(base, text)
#     final = remove_redundancy(refined)
#     return final


In [25]:
models = {
    "Lead-3": lead3_sum,
    "TextRank": textrank_sum,
    "BERTSUM": bertsum_sum,
    "PEGASUS": pegasus_sum,
    "BART": bart_sum,
    "T5": t5_sum,
    "Long-T5": longt5_sum,
    "LED": led_sum,
    "HIBERT": hibert_sum,
    "LLM": llm_sum,
    "CLARION-SUM": clarion
}

rows = []
for m, s in models.items():
  if callable(s):
    s = s(sample_doc)

  if not isinstance(s, str):
    s = str(s)
  rows.append([
        m,
        rouge_l(s, gold_summary),
        bert(s, gold_summary),
        factscore(s, sample_doc),
        redundancy(s)
    ])

df = pd.DataFrame(rows, columns=[
    "Model", "ROUGE-L ↑", "BERTScore ↑", "FactScore ↑", "Redundancy ↓"
])

df


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:949: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.poo

,Model,ROUGE-L ↑,BERTScore ↑,FactScore ↑,Redundancy ↓
0,Lead-3,0.226087,0.875127,1.000000,0.270998
1,TextRank,0.226087,0.875127,1.000000,0.270998
2,BERTSUM,0.300000,0.883385,1.000000,0.225772
3,PEGASUS,0.290323,0.890442,1.000000,0.000000
4,BART,0.121951,0.856325,0.948718,0.226887
5,T5,0.163934,0.862686,0.869565,0.313007
6,Long-T5,0.121951,0.856325,0.948718,0.226887
7,LED,0.121951,0.856325,0.948718,0.226887
8,HIBERT,0.300000,0.883385,1.000000,0.225772
9,LLM,0.121951,0.856325,0.948718,0.226887
